<a href="https://colab.research.google.com/github/Maagnitude/wikimovieplot-nlp-model/blob/main/wikimovieplot-nlp-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3η Εργασία** στο μάθημα **Μηχανική Μάθηση και Εφαρμογές**

# **Τμήμα Πληροφορικής και Τηλεματικής - Χαροκόπειο Πανεπιστήμιο**

# **Καζάζης Γεώργιος - it214124**

Στην παρούσα εργασία θα χρησιμοποιήσουμε μεθόδους επεξεργασίας φυσικής γλώσσας.

## **Modules**
Κάνουμε import τις απαραίτητες βιβλιοθήκες:
* **pandas** και **numpy** για την διαχείριση των δεδομένων μας.
* **re** για την επεξεργασία κειμένου (lowercase κτλ.)
* **tensorflow**, και από αυτήν το **keras** καθώς και τα **layers**, **losses**, **preprocessing** για την ανάπτυξη Νευρωνικών Δικτύων, και την εκπαίδευση τους.
* **TextVectorization** για να κάνουμε **vectorize** τα **Plot** και **Title** για την χρήση τους στην εκπαίδευση.
* Τα **warnings** για να τα φιλτράρουμε, ώστε να μην εμφανίζονται 

In [300]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import losses
from keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import warnings
warnings.filterwarnings(action='ignore')

##**Dataset**
Περνάμε το link από το **public github repo** μου, για το **dataset** μας, σε μορφή **csv**, το οποίο θα δώσουμε στην συνάρτηση για το φόρτωμα των δεδομένων μας.

In [299]:
url = "https://raw.githubusercontent.com/Maagnitude/wikimovieplot-nlp-model/main/dataset.csv"

## **Text preprocessing**
Με αυτή την συνάρτηση θα επεξεργαστούμε τα Title και Plot, ώστε να έχουν όλα πεζά γράμματα, να μην έχουν ειδικούς χαρακτήρες και σημεία στίξης.

In [301]:
def preprocess_text(text):
    text = text.lower() # convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text)
    return text

## **Βασική συνάρτηση preprocessing**
Εδώ υλοποιείται το μεγαλύτερο κομμάτι του notebook. 

Αρχικά φορτώνεται το dataset, ανακατεύονται οι γραμμές του με την sample, χωρίζεται σε train, validate και test set (έχοντας υπολογίσει τα ποσοστά του καθενός).

Εν συνεχεία, γίνεται **One-hot encoding** στα features '**Origin/Ethnicity**' και '**Genre**' με την χρήση της **pd.get_dummies** (σε κάθε **set**), τα οποία περνιούνται με τα κατάλληλα **keys** στα **dictionaries** που δημιουργούμε στο επόμενο βήμα. Από το shape[1] ενός εκ των τριών genre_dummies, παίρνουμε τις 20 διαφορετικές τιμές του feature 'Genre' ώστε να το χρησιμοποιήσουμε αργότερα.

Πριν περάσουμε τα '**Title**' και '**Plot**' στα **dictionaries**, τα περνάμε από την παραπάνω συνάρτηση επεξεργασίας κειμένου με την χρήση της **apply()**. 

Δημιουργούμε τα **dictionaries** και ύστερα τα περνάμε με την **from_tensor_slices()** (των **tf.data.Dataset**) στα Dataset object (**raw_train_ds**, **raw_val_ds**, **raw_test_ds**), και τέλος τα επιστρέφουμε. 

In [302]:
def load_data_wiki(file_path, batch_size=32, p_train=0.65, p_val=0.15):
    
    df = pd.read_csv(url)
    df = df.sample(frac=1, random_state=42)

    n = df.shape[0]
    n_train = int(n * p_train)
    n_val = int(n * p_val)
    n_test = n - n_train - n_val

    df_train, df_val, df_test = df.iloc[:n_train], df.iloc[n_train:n_train+n_val],\
                                                          df.iloc[n_train+n_val:]

    origin_dummies1 = pd.get_dummies(df_train['Origin/Ethnicity'])
    genre_dummies1 = pd.get_dummies(df_train['Genre'])

    origin_dummies2 = pd.get_dummies(df_val['Origin/Ethnicity'])
    genre_dummies2 = pd.get_dummies(df_val['Genre'])

    origin_dummies3 = pd.get_dummies(df_test['Origin/Ethnicity'])
    genre_dummies3 = pd.get_dummies(df_test['Genre'])

    n_labels = genre_dummies1.shape[1]

    df['Title'] = df['Title'].apply(preprocess_text)
    df['Plot'] = df['Plot'].apply(preprocess_text)  
                                                            
    train_data = {'Title': df_train['Title'].values,
                  'Origin': origin_dummies1.values.astype(np.float64),
                  'Plot': df_train['Plot'].values,
                  'Genre': genre_dummies1.values.astype(np.float64)}

    val_data = {'Title': df_val['Title'].values,
                'Origin': origin_dummies2.values.astype(np.float64),
                'Plot': df_val['Plot'].values,
                'Genre': genre_dummies2.values.astype(np.float64)}            

    test_data = {'Title': df_test['Title'].values,
                 'Origin': origin_dummies3.values.astype(np.float64),
                 'Plot': df_test['Plot'].values,
                 'Genre': genre_dummies3.values.astype(np.float64)}

    raw_train_ds = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
    raw_val_ds = tf.data.Dataset.from_tensor_slices(val_data).batch(batch_size)
    raw_test_ds = tf.data.Dataset.from_tensor_slices(test_data).batch(batch_size)

    return raw_train_ds, raw_val_ds, raw_test_ds, n_labels

In [335]:
raw_train_ds, raw_val_ds, raw_test_ds, n_labels = load_data_wiki(url)

## **Έλεγχος των δεδομένων**
Εδώ θα τσεκάρουμε τον **αριθμό των διαφορετικών Genre**, καθώς και αν τα **Title**, **Genre**, **Origin** και **Plot**, της πρώτης ταινίας του πρώτου batch, έχουν περαστεί σωστά (**Title** και **Plot** επεξεργασμένα σωστά - **Origin** και **Genre** **one-hot encoded**)

In [320]:
n_labels

20

In [321]:
for movie in raw_train_ds.take(1):
  print('First movie, from the first batch:')
  print('Title (lowercase etc.): ', movie['Title'][0].numpy())
  print('Genre (One-hot): ', movie['Genre'][0].numpy())
  print('Origin (One-hot): ', movie['Origin'][0].numpy())
  print('Plot (lowercase etc.): ', movie['Plot'][0].numpy())


First movie, from the first batch:
Title (lowercase etc.):  b'creature with the atom brain'
Genre (One-hot):  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Origin (One-hot):  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Plot (lowercase etc.):  b'a hulking zombie breaks into a mansion and kills a gangster named hennesy the blood stains left behind at the crime scene are radioactive and the fingerprints of the killer are of a man who had died days before the murder the police are baffled gangster boss frank buchanan who had been forced to flee the united states before he was deported was betrayed by members of his own underworld gang while traveling in europe he finds exnazi scientist wilhelm steigg gaye who is trying to reanimate the dead in order to provide a menial labor pool that is easily exploited buchanan funds the research and brings the scientist to america with the unstated goal of sending steiggs zombies out to murder those who oust

## **Πρώτο Vectorization**
Εδώ θα υλοποιήσουμε μία διαδικασία η οποία κάνει vectorize τα **Title** και **Plot**, ώστε να δημιουργηθεί μια δυαδική αναπαράσταση των λέξεων (**bag of words**) με λεξιλόγιο (αριθμό λέξεων) που θα ορίσουμε εμείς.

* Το πρώτο λεξιλόγιο θα έχει **500 λέξεις** (**max_features**). 
* Το **output** θα είναι **binary** (1 αν υπάρχει η λέξη, 0 αν δεν υπάρχει)
* Με το **pad_to_max_tokens**, βάζοντας το **True**, του λέμε να συμπληρώσει με **μηδενικά** έως ότου οι λέξεις να είναι όσο το **max_features** που θα του δώσουμε.

In [366]:
max_features = 500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

In [367]:
def vectorize_text(text):
  return vectorize_layer(tf.expand_dims(text, -1))

# **Χρήση του Title**
Για αρχή χρησιμοποιούμε μόνο το **title** ως είσοδο (χωρίς το Plot), και το δίνουμε όπως και το κάνουμε **adapt** στο **vectorize_layer**, ώστε παρακάτω να γίνει **vectorized** πριν δωθεί στα τελικά sets.

In [368]:
def final_sets(raw_train_ds, raw_val_ds, raw_test_ds):
    train_titles = raw_train_ds.map(lambda x: x['Title'])
    vectorize_layer.adapt(train_titles)
    val_titles = raw_val_ds.map(lambda x: x['Title'])
    vectorize_layer.adapt(val_titles)
    test_titles = raw_test_ds.map(lambda x: x['Title'])
    vectorize_layer.adapt(test_titles)

    train_ds = raw_train_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))
    val_ds = raw_val_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))
    test_ds = raw_test_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))

    return train_ds, val_ds, test_ds

In [369]:
train_ds, val_ds, test_ds = final_sets(raw_train_ds, raw_val_ds, raw_test_ds)

## **Πρώτο μοντέλο** 
Γίνεται η εκπαίδευση του πρώτου μοντέλου με χρήση μόνο του **Title**, για την πρόβλεψη του **Genre**.

**10020 παράμετροι** - **Είσοδος = 500**,  **Έξοδος = 20**

In [370]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 20)                10020     
                                                                 
Total params: 10,020
Trainable params: 10,020
Non-trainable params: 0
_________________________________________________________________


## **Αποτελέσματα**
Χρησιμοποιήθηκε ως **optimizer** η μέθοδος **Adam**, με **learning rate** **0.001**, και **default** οι υπόλοιπες παράμετροι.

Στις 10 εποχές είχαμε **Accuracy: 0.303**, και **Loss: 0.161**

Στις 20 εποχές είχαμε **Accuracy: 0.316**, και **Loss: 0.158**

**Δεν δοκιμάστηκε** σε παραπάνω εποχές μιας και θα το ξανατρέξουμε για **λεξιλόγιο 10000 λέξεων** ώστε να δούμε αν υπάρχει κάποια διαφορά. 

In [371]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/20
468/468 [==============================] - 4s 4ms/step - loss: 0.5053 - accuracy: 0.1596 - val_loss: 0.3737 - val_accuracy: 0.2728
Epoch 2/20
468/468 [==============================] - 2s 4ms/step - loss: 0.3065 - accuracy: 0.2937 - val_loss: 0.2614 - val_accuracy: 0.2757
Epoch 3/20
468/468 [==============================] - 3s 6ms/step - loss: 0.2326 - accuracy: 0.2919 - val_loss: 0.2144 - val_accuracy: 0.2751
Epoch 4/20
468/468 [==============================] - 2s 4ms/step - loss: 0.1992 - accuracy: 0.2955 - val_loss: 0.1914 - val_accuracy: 0.2789
Epoch 5/20
468/468 [==============================] - 2s 3ms/step - loss: 0.1817 - accuracy: 0.3036 - val_loss: 0.1788 - val_accuracy: 0.2847
Epoch 6/20
468/468 [==============================] - 2s 4ms/step - loss: 0.1718 - accuracy: 0.3094 - val_loss: 0.1715 - val_accuracy: 0.2902
Epoch 7/20
468/468 [==============================] - 3s 6ms/step - loss: 0.1657 - accuracy: 0.3146 - val_loss: 0.1671 - val_accuracy: 0.2939
Epoch 

## **Λεξιλόγιο 10000 λέξεων**
Αυξάνουμε τις λέξεις στο **bag of words**, και ξαναδημιουργούμε τα **sets**. Ύστερα εκπαιδεύουμε πάλι το μοντέλο μας και το τρέχουμε με τις **ίδιες παραμέτρους** για **20 εποχές**, για να το συγκρίνουμε με το **πρώτο**.

Πλέον, έχουμε **200020 παραμέτρους**, **Είσοδο=10000**, **Έξοδο=20**.

In [375]:
max_features = 10000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets(raw_train_ds, raw_val_ds, raw_test_ds)

In [376]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 20)                200020    
                                                                 
Total params: 200,020
Trainable params: 200,020
Non-trainable params: 0
_________________________________________________________________


## **Αποτελέσματα 2**
Στις **20 εποχές** έχουμε **Accuracy: 0.355** και **Loss: 0.159**.


**Απάντηση στην ερώτηση**:

Τα έχει πάει σίγουρα καλύτερα από το πρώτο, αλλά μεγάλη διαφορά δεν υπάρχει παρ' όλο που αυξήθηκαν οι λέξεις στο λεξιλόγιο, κι αυτό οφείλεται, κατά πάσα πιθανότητα, στο ότι με τον τίτλο μόνο, το μοντέλο δεν μπορεί να πάρει τις απαραίτητες πληροφορίες στην εκπαίδευση του. 

In [377]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/20
468/468 [==============================] - 5s 9ms/step - loss: 0.5180 - accuracy: 0.2409 - val_loss: 0.3915 - val_accuracy: 0.2925
Epoch 2/20
468/468 [==============================] - 4s 9ms/step - loss: 0.3250 - accuracy: 0.3160 - val_loss: 0.2779 - val_accuracy: 0.2896
Epoch 3/20
468/468 [==============================] - 4s 8ms/step - loss: 0.2472 - accuracy: 0.3210 - val_loss: 0.2274 - val_accuracy: 0.2948
Epoch 4/20
468/468 [==============================] - 4s 9ms/step - loss: 0.2095 - accuracy: 0.3326 - val_loss: 0.2011 - val_accuracy: 0.3035
Epoch 5/20
468/468 [==============================] - 5s 10ms/step - loss: 0.1885 - accuracy: 0.3472 - val_loss: 0.1860 - val_accuracy: 0.3136
Epoch 6/20
468/468 [==============================] - 4s 9ms/step - loss: 0.1755 - accuracy: 0.3582 - val_loss: 0.1767 - val_accuracy: 0.3186
Epoch 7/20
468/468 [==============================] - 4s 8ms/step - loss: 0.1669 - accuracy: 0.3700 - val_loss: 0.1707 - val_accuracy: 0.3229
Epoch

# **Χρήση του Plot**
Τώρα με την χρήση της συνάρτησης **final_sets_plot**, θα δημιουργήσουμε νέα sets τα οποία θα έχουν μόνο το **Plot**, ώστε να δοκιμάσουμε να εκπαιδεύσουμε ένα μοντέλο με τα **σενάρια των ταινιών** και να ελέγξουμε τα αποτελέσματα του.

In [378]:
def final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds):
    train_titles = raw_train_ds.map(lambda x: x['Plot'])
    vectorize_layer.adapt(train_titles)
    val_titles = raw_val_ds.map(lambda x: x['Plot'])
    vectorize_layer.adapt(val_titles)
    test_titles = raw_test_ds.map(lambda x: x['Plot'])
    vectorize_layer.adapt(test_titles)

    train_ds = raw_train_ds.map(lambda x: (vectorize_text(x['Plot']), x['Genre']))
    val_ds = raw_val_ds.map(lambda x: (vectorize_text(x['Plot']), x['Genre']))
    test_ds = raw_test_ds.map(lambda x: (vectorize_text(x['Plot']), x['Genre']))

    return train_ds, val_ds, test_ds

Θα χρησιμοποιήσουμε ένα **λεξιλόγιο 1000 λέξεων**, και όλες τις υπόλοιπες παραμέτρους ίδιες.

In [379]:
max_features = 1000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

**Παράμετροι=20020**, **Είσοδος=1000**, **Έξοδος=20**

In [380]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 20)                20020     
                                                                 
Total params: 20,020
Trainable params: 20,020
Non-trainable params: 0
_________________________________________________________________


## **Αποτελέσματα 3**
Έχουμε **καθαρή βελτίωση** από τα προηγούμενα, μιας και πλέον έχει ως είσοδο τα **σενάρια** των ταινιών από τα οποία μπορεί να αντλήσει σαφώς περισσότερες πληροφορίες, κι έτσι με **λεξιλόγιο 1000 λέξεων** τα πήγε πολύ καλύτερα από ότι τα είχε πάει το μοντέλο που εκπαιδεύτηκε στους **τίτλους** των ταινιών με **λεξιλόγιο 10000 λέξεων**.

**Accuracy: 0.454**, **Loss: 0.139**

Παρατηρούμε όμως ότι μετά την **7η εποχή** το **val_loss** μόνο αυξάνεται και γενικά ενώ το μοντέλο πετυχαίνει **accuracy** στο train set τιμές υψηλότερες του **0.57**, δεν γενικεύει αποτελεσματικό, περίπτωση **υπερεκπαίδευσης**. Θα γίνει μία δοκιμή και στις **10 εποχές**.

In [381]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/20
468/468 [==============================] - 3s 6ms/step - loss: 0.2020 - accuracy: 0.3591 - val_loss: 0.1631 - val_accuracy: 0.4217
Epoch 2/20
468/468 [==============================] - 4s 9ms/step - loss: 0.1463 - accuracy: 0.4523 - val_loss: 0.1432 - val_accuracy: 0.4431
Epoch 3/20
468/468 [==============================] - 3s 5ms/step - loss: 0.1302 - accuracy: 0.4830 - val_loss: 0.1367 - val_accuracy: 0.4553
Epoch 4/20
468/468 [==============================] - 3s 6ms/step - loss: 0.1221 - accuracy: 0.5022 - val_loss: 0.1342 - val_accuracy: 0.4608
Epoch 5/20
468/468 [==============================] - 3s 6ms/step - loss: 0.1168 - accuracy: 0.5181 - val_loss: 0.1332 - val_accuracy: 0.4616
Epoch 6/20
468/468 [==============================] - 3s 5ms/step - loss: 0.1130 - accuracy: 0.5350 - val_loss: 0.1329 - val_accuracy: 0.4625
Epoch 7/20
468/468 [==============================] - 4s 9ms/step - loss: 0.1099 - accuracy: 0.5466 - val_loss: 0.1330 - val_accuracy: 0.4610
Epoch 

## **Εκπαίδευση με 10 εποχές για αποφυγή υπερεκπαίδευσης**
Πράγματι έχουμε **Accuracy: 0.463**, και **Loss: 0.131**. Υπάρχει μικρή βελτίωση, όχι αξιοσημείωτη.

In [385]:
max_features = 1000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

In [387]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 20)                20020     
                                                                 
Total params: 20,020
Trainable params: 20,020
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
468/468 [==============================] - 3s 6ms/step - loss: 0.2039 - accuracy: 0.3495 - val_loss: 0.1635 - val_accuracy: 0.4196
Epoch 2/10
468/468 [==============================] - 3s 7ms/step - loss: 0.1466 - accuracy: 0.4502 - val_loss: 0.1437 - val_accuracy: 0.4451
Epoch 3/10
468/468 [==============================] - 2s 5ms/step - loss: 0.1304 - accuracy: 0.4828 - val_loss: 0.1370 - val_accuracy: 0.4532
Epoch 4/10
468/468 [==============================] - 2s 5ms/step - loss: 0.1222 - accuracy: 0.5027 - val_loss: 0.1343 - val_accuracy: 0.4584
Epoch 5/1

In [390]:
max_features = 30000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

**Παράμετροι=600020**, **Είσοδος=30000**, **Έξοδος=20**

In [391]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 20)                600020    
                                                                 
Total params: 600,020
Trainable params: 600,020
Non-trainable params: 0
_________________________________________________________________


## **Αποτελέσματα 4**
Στις **20 εποχές** έχουμε **Accuracy: 0.502** και **Loss: 0.199**

Στις **10 εποχές** έχουμε **Accuracy: 0.490** και **Loss: 0.290**

Στις **5 εποχές** έχουμε **Accuracy: 0.499** και **Loss: 0.238**

Οπότε στις 20 εποχές (που έχουν τρέξει και τα υπόλοιπα) είχαμε το καλύτερο αποτέλεσμα, το οποίο γενικά ίσως θεωρείται χαμηλό σαν accuracy, αλλά είναι σαφώς καλύτερο από τυχαία επιλογή ανάμεσα σε 20 κλάσεις (όπως είπατε).

In [396]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/10
468/468 [==============================] - 10s 20ms/step - loss: 0.0094 - accuracy: 0.9874 - val_loss: 0.2492 - val_accuracy: 0.4874
Epoch 2/10
468/468 [==============================] - 9s 19ms/step - loss: 0.0089 - accuracy: 0.9886 - val_loss: 0.2560 - val_accuracy: 0.4857
Epoch 3/10
468/468 [==============================] - 8s 17ms/step - loss: 0.0083 - accuracy: 0.9892 - val_loss: 0.2623 - val_accuracy: 0.4854
Epoch 4/10
468/468 [==============================] - 9s 20ms/step - loss: 0.0078 - accuracy: 0.9900 - val_loss: 0.2682 - val_accuracy: 0.4848
Epoch 5/10
468/468 [==============================] - 9s 20ms/step - loss: 0.0073 - accuracy: 0.9909 - val_loss: 0.2739 - val_accuracy: 0.4828
Epoch 6/10
468/468 [==============================] - 8s 17ms/step - loss: 0.0069 - accuracy: 0.9913 - val_loss: 0.2794 - val_accuracy: 0.4802
Epoch 7/10
468/468 [==============================] - 9s 20ms/step - loss: 0.0065 - accuracy: 0.9916 - val_loss: 0.2847 - val_accuracy: 0.480

## **Ενσωματώσεις λέξεων**

In [399]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(20)])

model.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          480016    
                                                                 
 dropout_2 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_38 (Dense)            (None, 20)                340       
                                                                 
Total params: 480,356
Trainable params: 480,356
Non-trainable params: 0
_______________________________________________

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))
epochs = 5
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)